# PixelCNN

In [1]:
# import matplotlib.pyplot as plt

# import torch
# from torch import nn, optim
# from torch.utils.data import DataLoader

# import torchvision
# import torchvision.transforms as T

In [2]:
# # parameters from the paper
# batch_size = 16
# num_layers = 7
# num_channels = 32

# # other parameters
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# lr = 0.001
# num_epochs = 10

The architecture is designed for MNIST only and for static inputs.

In [3]:
# architecture
# 1. 7 x 7 with conv mask A
# 2. stack 3 x 3 with conv mask B
# 3. ReLU + 1x1 conv layer with mask B (2 layers)
# 4. 256 Ways softmax

In [4]:
# # Type A or B masked Conv2d 
# class MaskedConv2d(nn.Module):
#     def __init__(self, in_channels, out_channels, kernel_size, mask='A'): 
#         super().__init__()
#         # calculate the padding automatically 
#         # so that input width+height dims = output dims
#         padding = int((kernel_size - 1) / 2)
        
#         self.conv = nn.Conv2d(in_channels=in_channels,
#                              out_channels=out_channels,
#                              kernel_size=kernel_size,
#                              padding=padding)
        
#         # calculate masks A or B as in the original paper
#         mask_idxs = torch.arange(0, kernel_size**2).view(kernel_size, kernel_size)
#         if mask == 'A':
#             mask = (mask_idxs < padding * kernel_size + padding).float().to(device)
#         elif mask == 'B':
#             mask = (mask_idxs <= padding * kernel_size + padding).float().to(device)
#         self.register_buffer('mask', mask)
            
#     def forward(self, x):
#         # apply mask, see below link for more info
#         # https://discuss.pytorch.org/t/applying-custom-mask-on-kernel-for-cnn/87099
        
#         with torch.no_grad():
#             self.conv.weight = nn.Parameter(self.conv.weight * self.mask)
#         return self.conv(x)

In [5]:
# test2dLayer = MaskedConv2d(in_channels=1, out_channels=num_channels, kernel_size=7).to(device)
# testdata = torch.randn(1, 1, 28, 28, device=device)
# test2dLayer(testdata).shape

In [6]:
# class PixelCNN(nn.Module):
    
#     def __init__(self):
#         super().__init__()
#         self.layers = nn.Sequential(
#                 MaskedConv2d(in_channels=1, out_channels=num_channels, kernel_size=7, mask='A'), 
#                 nn.ReLU(),
#                 MaskedConv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=3, mask='B'), 
#                 nn.ReLU(),
#                 MaskedConv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=3, mask='B'), 
#                 nn.ReLU(),
#                 nn.Conv2d(num_channels, num_channels*2, kernel_size=1),
#                 nn.ReLU(),
#                 nn.Conv2d(num_channels*2, 1, kernel_size=1)
#         )
    
#     def forward(self, x):
#         return self.layers(x)

In [7]:
# test_images = torch.randn(batch_size, 1, 28, 28, device=device)
# model = PixelCNN().to(device)
# model(test_images).shape

In [8]:
# train_dataset = torchvision.datasets.MNIST(root='../datasets/', 
#                                            train=True, 
#                                            transform=T.ToTensor(), 
#                                            download=True)

# test_dataset = torchvision.datasets.MNIST(root='../datasets/', 
#                                            train=False, 
#                                            transform=T.ToTensor(), 
#                                            download=False)

In [9]:
# train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)
# test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=2)

In [10]:
# model = PixelCNN().to(device)
# # mse would most likely also work
# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model.parameters())

In [11]:
# def train():
#     for epoch in range(1, num_epochs+1):
#         train_loss = []
#         for features, _ in train_dataloader:
#             features = features.to(device)
#             logits = model(features)
#             loss = criterion(logits, features)
            
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#             train_loss.append(loss.cpu().item())
        
#         print(f'Epoch: {epoch}/{num_epochs}, Loss: {sum(train_loss)/len(train_loss)}')

In [12]:
# train()

In [13]:
# # test the reconstruction by comparing original images with the reconstructed images
# with torch.inference_mode():
#     for idx in range(5):
#         image, _ = test_dataset[idx]
#         image = image.unsqueeze(0)
#         generated_image = model(image.to(device))
#         generated_image = torch.sigmoid(generated_image)
#         generated_image = generated_image.squeeze()

#         plt.subplot(1, 2, 1)
#         plt.imshow(image.squeeze(), cmap="gray")
#         plt.title("Original")
#         plt.axis("off")

#         plt.subplot(1, 2, 2)
#         plt.imshow(generated_image.cpu(), cmap="gray")
#         plt.title("Reconstructed")
#         plt.axis("off")
#         plt.show()

In [14]:
# image, _ = test_dataset[0]
# image = image.squeeze()
# for row_idx, row in enumerate(image):
#     for col_idx, _ in enumerate(image):
#         if row_idx >= 14:
#             image[row_idx][col_idx] = 0
# plt.imshow(image.squeeze())

In [15]:
# image, _ = test_dataset[0]
# image = image.squeeze()
# for row_idx, row in enumerate(image):
#     for col_idx, _ in enumerate(image):
#         if row_idx >= 14:
#             image[row_idx][col_idx] = 0

# image = image.unsqueeze(0).unsqueeze(0).to(device) 
# with torch.inference_mode():
#     for row_idx in range(28):
#         for col_idx in range(28):
#             generated_image = model(image)
#             generated_image = torch.sigmoid(generated_image)
#             generated_image = generated_image.squeeze()
#             if row_idx >=14:
#                 image[0][0][row_idx][col_idx] = generated_image[row_idx][col_idx]
            
# plt.imshow(image.cpu().squeeze(), cmap="gray")
# plt.title("Reconstructed")
# plt.axis("off")
# plt.show()


## PixelCNN with Gated Convolutions

This part is highly inspired by [uvadlc-notebooks](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial12/Autoregressive_Image_Modeling.html)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision
import torchvision.transforms as T
from torchvision.datasets import MNIST

In [51]:
# hyperparameters
NUM_EPOCHS = 10
BATCH_SIZE = 128
NUM_CHANNELS = 64
NUM_LAYERS = 7
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [64]:
train_dataset = MNIST(root='../datasets/', train=True, download=True, transform=T.ToTensor())
test_dataset = MNIST(root='../datasets/', train=False, download=False, transform=T.ToTensor())

In [68]:
train_dataloader = data.DataLoader(dataset=train_dataset, 
                                   batch_size=BATCH_SIZE, 
                                   shuffle=True, 
                                   num_workers=2,
                                   drop_last=True)
test_dataloader = data.DataLoader(dataset=test_dataset, 
                                   batch_size=BATCH_SIZE, 
                                   shuffle=True, 
                                   num_workers=2)

In [43]:
class MaskedConvolution(nn.Module):
    
    def __init__(self, in_channels, out_channels, mask):
        super().__init__()
        kernel_size = mask.shape
        padding = tuple([(size-1)//2 for size in kernel_size])
        self.conv = nn.Conv2d(in_channels=in_channels, 
                              out_channels=out_channels, 
                              kernel_size=kernel_size,
                              padding=padding)
        
        self.register_buffer('mask', mask)
    
    def forward(self, x):
        with torch.no_grad():
            self.conv.weight = nn.Parameter(self.conv.weight * self.mask)
        return self.conv(x)

In [44]:
mask = torch.rand(3, 3)
img = torch.randn(1, 1, 28, 28)
mask_conv = MaskedConvolution(1, 32, mask)
mask_conv(img).shape

torch.Size([1, 32, 28, 28])

In [45]:
class VerticalStackConvolution(MaskedConvolution):
    def __init__(self, in_channels, out_channels, kernel_size=3, mask_type='B'):
        assert mask_type in ['A', 'B']
        mask = torch.ones(kernel_size, kernel_size)
        mask[kernel_size//2+1:,:] = 0
        if mask_type=='A':
            mask[kernel_size//2,:] = 0
        super().__init__(in_channels, out_channels, mask)
        
class HorizontalStackConvolution(MaskedConvolution):
    def __init__(self, in_channels, out_channels, kernel_size=3, mask_type='B'):
        assert mask_type in ['A', 'B']
        mask = torch.ones(1, kernel_size)
        mask[0, kernel_size//2+1:] = 0
        if mask_type=='A':
            mask[0, kernel_size//2] = 0
        super().__init__(in_channels, out_channels, mask)

In [46]:
test = VerticalStackConvolution(1, 32, 3)
print(test.mask)
test = VerticalStackConvolution(1, 32, 3, 'A')
print(test.mask)
test = HorizontalStackConvolution(1, 32, 3)
print(test.mask)
test = HorizontalStackConvolution(1, 32, 3, 'A')
print(test.mask)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 0.]])
tensor([[1., 0., 0.]])


In [47]:
class GatedConvolution(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.v = VerticalStackConvolution(in_channels=in_channels,
                                                   out_channels=in_channels*2)
        
        self.h = HorizontalStackConvolution(in_channels=in_channels,
                                                   out_channels=in_channels*2)
        
        self.v_to_h = nn.Conv2d(in_channels=in_channels*2,
                                     out_channels=in_channels*2,
                                     kernel_size=1)
        
        self.h_out = nn.Conv2d(in_channels=in_channels,
                                      out_channels=in_channels,
                                      kernel_size=1)
        
    def forward(self, v_prev, h_prev):
        v = self.v(v_prev)
        v_f, v_g = torch.chunk(v, chunks=2, dim=1)
        v_out = torch.tanh(v_f) * torch.sigmoid(v_g)
        
        h = self.h(h_prev)
        h += self.v_to_h(v)
        h_f, h_g = torch.chunk(h, chunks=2, dim=1)
        h_out = torch.tanh(h_f) * torch.sigmoid(h_g)
        h_out = self.h_out(h_out)
        h_out += h_prev
        
        return v_out, h_out

In [48]:
# test gated convolution
gated = GatedConvolution(NUM_CHANNELS)
v = torch.randn(1, NUM_CHANNELS, 28, 28)
h = torch.randn(1, NUM_CHANNELS, 28, 28)
v, h = gated(v, h)
print(v.shape, h.shape)

torch.Size([1, 64, 28, 28]) torch.Size([1, 64, 28, 28])


In [106]:
class GatedPixelCNN(nn.Module):
    def __init__(self, hidden_channels, num_gated):
        super().__init__()
        self.v = VerticalStackConvolution(in_channels=1,
                                          out_channels=hidden_channels,
                                          mask_type='A')
        self.h = HorizontalStackConvolution(in_channels=1,
                                          out_channels=hidden_channels,
                                          mask_type='A')
        
        self.gated_convolutions = nn.ModuleList([GatedConvolution(hidden_channels) 
                                                 for _ in range(num_gated)])
        # later we apply a 256 way softmax
        self.output = nn.Conv2d(in_channels=hidden_channels, 
                                out_channels=256,
                                kernel_size=1)
    def forward(self, x):
        v = self.v(x)
        h = self.h(x)
        
        for gated_layer in self.gated_convolutions:
            v, h = gated_layer(v, h)
        out = self.output(F.relu(h))
        # from Batch, Classes, Height, Width to Batch, Classes, Channel, Height, Width
        out = out.unsqueeze(dim=2)
        return out

In [110]:
# test pixelcnn
gated_pixel_cnn = GatedPixelCNN(NUM_CHANNELS, NUM_LAYERS)
img = torch.randn(1, 1, 28, 28)
labels = (torch.rand(1, 1, 28, 28) * 256).long()
output = gated_pixel_cnn(img)
print(output.shape, labels.shape)
test = F.cross_entropy(output, labels, reduction='mean')
print(test)

torch.Size([1, 256, 1, 28, 28]) torch.Size([1, 1, 28, 28])
tensor(5.5776, grad_fn=<NllLoss2DBackward0>)
